In [7]:
#### to do list
# 1. Bet --- DONE
# 2. Volume Removal ---- DONE
# 3. Reallignment ---- DONE
# 4. SliceTime Correction
# 5. Coregistration --- DONE
# 6. Segmentation ---- DONE
# 7. Warping  ---- DONE
# 8. Smoothing ---- DONE
# 9. ART removal ---- DONE
# 10. First Level
# 11. Flame
# 12. Datagrabber  ---- DONE
# 13. Datasink     ---- DONE

In [8]:
from nipype import Node,Workflow,IdentityInterface
from nipype.interfaces import fsl
from nipype.interfaces.io import  SelectFiles, DataSink
from nipype.algorithms import rapidart
    

mni = "/home/paradeisios/Desktop/fsl_test_dir/MNI152_T1_2mm.nii.gz" # substitute with your mni whole brain template
mni_brain = "/home/paradeisios/Desktop/fsl_test_dir/MNI152_T1_2mm_brain.nii.gz" # substitute with your mni extracted brain


data_dir  = "/home/paradeisios/Desktop/fsl_test_dir/data/" # substitute with your data dir
output_dir   = "/home/paradeisios/Desktop/fsl_test_dir/output/" # substitute with your output dir
subject_list = ["1"] # substitute with the number of subjects you have

In [9]:
##data grabber and data deposit


# Iterator - this loops though the subject your list provided in cell[2] - if you do not use standard bids, might need modification
infosource = Node(IdentityInterface(fields = ["subject_id"]),name="infosource")
infosource.iterables = [("subject_id", subject_list)]


# String template with {}-based strings - this is the DATA GRABBER - you have to provide a template as to how the data are organized in the data folder, so it can find the anatomical and the funcitonal scans
template = { "anat": data_dir  + "sub-{subject_id}/anat/T1_w.nii.gz",
             "func": data_dir  + "sub-{subject_id}/func/T2_w.nii.gz"}

selectfiles = Node(SelectFiles(template),name='selectfiles')

# Data sink - this is the handler where your data will be output --- IF YOU DO NOT SPECIFY WHICH DATA YOU WANT TO BE SAVED, THEY WILL NOT BE SAVED (this is already done in the workflow, you can change it to add more or less saved outputs)

datasink = Node(DataSink(),name="datasink")
datasink.inputs.base_directory = output_dir


In [10]:
##structural workflow

bet = Node(fsl.BET(),name="bet")
bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
bet.inputs.vertical_gradient = 0.2 #value to linearly improve estimations at the bottom of the brain -increasing might understimate top
bet.inputs.mask = True #creates a binary mask of the brain


segment = Node(fsl.FAST(),name="segment")
"""You are interested mainly in the restored version of the structural and 
    pve0 = grey matter
    pve1 = white matter
    pve2 = CSF """
segment.inputs.img_type = 1 # indicate its a T1 image
segment.inputs.number_classes = 3 # seperate structural into Grey Matter, White Matter and CSF
segment.inputs.output_biascorrected = True #return structural image bias field corrected
segment.inputs.output_biasfield = True #return bias field
segment.inputs.segments=True #create a binary mask for each tissue type
segment.inputs.no_pve=False #if you dont want partial volume estimation, set to true


flirt_struct = Node(fsl.FLIRT(),name="flirt_struct")
"""This step performs linear registration of the structural volume to the standard space"""
flirt_struct.inputs.reference = mni_brain 
flirt_struct.inputs.dof = 12 #use 12 dof in structural flirt to prepare image for FNIRT nonlinear normalization to mni space
flirt_struct.inputs.out_matrix_file = "h2s_affine.mat" #keep this to facilite struct and func normalization to mni space using FNIRT
flirt_struct.inputs.interp = "spline" # change to trilinear if you are mostly interested in non-subcortical normalization. Trilinear is faster, but might have some rotational issues
flirt_struct.inputs.cost = "mutualinfo"


fnirt_struct = Node(fsl.FNIRT(),name="fnirt_struct")
"""This step performs non-linear registration of the structural volume to the standard space"""
fnirt_struct.inputs.ref_file = mni
fnirt_struct.inputs.config_file = "T1_2_MNI152_2mm"
fnirt_struct.inputs.fieldcoeff_file = True  #keep the deformation fields to faciliate func normalization with FNIRT

In [11]:
##functional workflow
img_to_float = Node(fsl.ImageMaths(),name="img_to_float")
"""This converts the 4d in float representation to faciliate some further computations"""
img_to_float.inputs.out_data_type='float'
img_to_float.inputs.op_string=''


vol_removal = Node(fsl.ExtractROI(), name="vol_removal")
"""This step removes the 5 first volumes to avoid T1 saturation"""
vol_removal.inputs.t_min = 5 #number of scans to exclude from functional image
vol_removal.inputs.t_size = -1 # keep all the rest until the end


realign = Node(fsl.MCFLIRT(),name = "realign")
"""This step realigns data to the first image""" 
realign.inputs.save_mats = True #save realignment  parameters
realign.inputs.save_plots = True #save parameter plots
realign.inputs.dof = 6 #use this for rigid body correction


art = Node(rapidart.ArtifactDetect(),name="art")
"""This step performs artifact detection""" 
art.inputs.mask_type = "spm_global"
art.inputs.parameter_source = 'FSL'
art.inputs.norm_threshold = 1
art.inputs.use_differences = [True, False]
art.inputs.zintensity_threshold = 3


mean_img = Node(fsl.maths.MeanImage(),name= "mean_img")
mean_img.inputs.dimension = "T" # find mean image across the time dimension


flirt_func = Node(fsl.FLIRT(),name = "flirt_func")
"""This step performs linear registration of the functional volume to the structural volume. It is mainly used to
extract the transformation matrix to later register to standard space"""
flirt_func.inputs.dof = 6
flirt_func.inputs.out_matrix_file = "f2h_affine.mat"
flirt_func.inputs.interp = "spline"
flirt_func.inputs.cost = "mutualinfo"


func_warp = Node(fsl.ApplyWarp(),name = "func_warp")
"""This step performs non linear registration to standard space using the affine matrix extracted 
in the flirt above and the fnirt non linear warp in the structural pipeline """
func_warp.inputs.ref_file = mni


smooth = Node(fsl.IsotropicSmooth(),name="smooth")
"""This step performs smoothing"""
smooth.inputs.fwhm = 6 #change the smoothing kernel 

In [13]:
struct_flow = Workflow(name="struct_flow", base_dir="/home/paradeisios/Desktop/fsl_test_dir/output/")
struct_flow.connect(bet,"out_file",segment,"in_files")
struct_flow.connect(segment,"restored_image",flirt_struct,"in_file")
struct_flow.connect(flirt_struct,"out_matrix_file",fnirt_struct,"affine_file")

#########################################################################################################
func_flow = Workflow(name="func_flow", base_dir="/home/paradeisios/Desktop/fsl_test_dir/output/")
func_flow.connect(img_to_float,"out_file",vol_removal,"in_file")
func_flow.connect(vol_removal, "roi_file", realign, "in_file")
func_flow.connect(realign,"out_file",art,"realigned_files")
func_flow.connect(realign,"par_file",art,"realignment_parameters")
func_flow.connect(realign,"out_file",mean_img,"in_file")
func_flow.connect(mean_img,"out_file",flirt_func,"in_file")
func_flow.connect(flirt_func,"out_matrix_file",func_warp,"premat")
func_flow.connect(realign,"out_file",func_warp,"in_file")
func_flow.connect(func_warp,"out_file",smooth,"in_file")

########################################################################################################

preprocess_flow = Workflow(name="preprocess_flow",base_dir="/home/paradeisios/Desktop/fsl_test_dir/output/")

preprocess_flow.connect([(infosource,selectfiles,[("subject_id","subject_id")]),
                         (selectfiles,struct_flow,[("anat","bet.in_file")]),
                         (selectfiles,struct_flow,[("anat","fnirt_struct.in_file")]),
                         (selectfiles,func_flow,[("func","img_to_float.in_file")]),                                                   
                         (struct_flow,func_flow,[("segment.restored_image","flirt_func.reference")]),
                         (struct_flow,func_flow,[("fnirt_struct.fieldcoeff_file","func_warp.field_file")])])

#######################################################################################################
#######################################################################################################

#this is your datasink, where you stash the results you want

preprocess_flow.connect([(infosource,datasink,[("subject_id", "container")]),
                         #########################################################################
                         (struct_flow,datasink,[("bet.out_file","bet.@bet_brain")]),
                         (struct_flow,datasink,[("bet.mask_file","bet.@bet_mask")]),
                         (struct_flow,datasink,[("segment.restored_image","segment.@restored_image")]),
                         (struct_flow,datasink,[("segment.tissue_class_files","segment.@tissue_class_files")]),
                         (struct_flow,datasink,[("segment.bias_field","segment.@bias_field")]),
                         (struct_flow,datasink,[("segment.tissue_class_map","segment.@tissue_class_map")]),
                         (struct_flow,datasink,[("flirt_struct.out_file","flirt_struct.@structural_flirt")]),
                         (struct_flow,datasink,[("flirt_struct.out_matrix_file","flirt_struct.@affine_parameters")]),
                         (struct_flow,datasink,[("fnirt_struct.warped_file","flirt_struct.@structural_fnirt")]),
                         (struct_flow,datasink,[("fnirt_struct.field_file","flirt_struct.@fieldcoeff_file")]),
                         
                         #########################################################################
                         (func_flow,datasink,[("vol_removal.roi_file","vol_removal.@files")]),
                         (func_flow,datasink,[("realign.out_file","realign.@realigned_files")]),
                         (func_flow,datasink,[("realign.par_file","realign.@realig_parameters")]),
                         (func_flow,datasink,[("art.outlier_files","art.@outlier_files")]),
                         (func_flow,datasink,[("art.plot_files","art.@plot_files")]),
                         (func_flow,datasink,[("art.displacement_files","art.@displacement_files")]),
                         (func_flow,datasink,[("mean_img.out_file","mean_img.@mean_img")]),
                         (func_flow,datasink,[("flirt_func.out_matrix_file","flirt_func.@affine_parameters")]),
                         (func_flow,datasink,[("flirt_func.out_file","flirt_func.@functional_flirt")]),
                         (func_flow,datasink,[("func_warp.out_file","func_warp.@functional_fnirt")]),
                         (func_flow,datasink,[("smooth.out_file","smooth.@smoothed_files")])])

In [15]:
preprocess_flow.write_graph("workflow_graph.dot")

preprocess_flow.write_graph(graph2use='colored')
preprocess_flow.write_graph(graph2use='flat')

210505-10:44:38,127 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/fsl_test_dir/output/preprocess_flow/workflow_graph.png (graph2use=hierarchical, simple_form=True).
210505-10:44:39,257 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/fsl_test_dir/output/preprocess_flow/graph.png (graph2use=colored, simple_form=True).
210505-10:44:42,451 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/fsl_test_dir/output/preprocess_flow/graph.png (graph2use=flat, simple_form=True).


'/home/paradeisios/Desktop/fsl_test_dir/output/preprocess_flow/graph.png'

In [16]:
preprocess_flow.run(plugin='MultiProc', plugin_args={'n_procs' : 4})

210505-10:46:20,978 nipype.workflow INFO:
	 Workflow preprocess_flow settings: ['check', 'execution', 'logging', 'monitoring']
210505-10:46:20,991 nipype.workflow INFO:
	 Running in parallel.
210505-10:46:20,995 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 13.79/13.79, Free processors: 4/4.
210505-10:46:21,92 nipype.workflow INFO:
	 [Node] Setting-up "preprocess_flow.selectfiles" in "/home/paradeisios/Desktop/fsl_test_dir/output/preprocess_flow/_subject_id_1/selectfiles".
210505-10:46:21,98 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210505-10:46:21,105 nipype.workflow INFO:
	 [Node] Finished "preprocess_flow.selectfiles".
210505-10:46:22,997 nipype.workflow INFO:
	 [Job 0] Completed (preprocess_flow.selectfiles).
210505-10:46:23,4 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 13.79/13.79, Free processors: 4/4.
210505-10:46:23,100 nipype.workflow INFO:
	 

210505-10:47:57,88 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 13.39/13.79, Free processors: 2/4.
                     Currently running:
                       * preprocess_flow.func_flow.art
                       * preprocess_flow.struct_flow.segment
210505-10:47:58,39 nipype.workflow INFO:
	 [Node] Finished "preprocess_flow.func_flow.art".
210505-10:47:59,87 nipype.workflow INFO:
	 [Job 5] Completed (preprocess_flow.func_flow.art).
210505-10:47:59,90 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 13.59/13.79, Free processors: 3/4.
                     Currently running:
                       * preprocess_flow.struct_flow.segment
210505-10:49:29,404 nipype.workflow INFO:
	 [Node] Finished "preprocess_flow.struct_flow.segment".
210505-10:49:31,179 nipype.workflow INFO:
	 [Job 7] Completed (preprocess_flow.struct_flow.segment).
210505-10:49:31,189 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks

210505-11:05:30,244 nipype.workflow INFO:
	 [Node] Finished "preprocess_flow.datasink".
210505-11:05:32,103 nipype.workflow INFO:
	 [Job 13] Completed (preprocess_flow.datasink).
210505-11:05:32,106 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 13.79/13.79, Free processors: 4/4.
